In [3]:
import sys
import math

def hextobin(s):
    l = len(s)
    convert = bin(int(s, 16))[2:]
    prefix = '0' * (len(s) * 4 - len(convert))
    return prefix + convert
assert hextobin('D2FE28') == '110100101111111000101000'

def parse_hex(s):
    packet = Packet(hextobin(s))
    res = packet.parse_bin()
    return res[0] # discard the length

assert hextobin('38006F45291200') == '00111000000000000110111101000101001010010001001000000000'

In [7]:
class Packet():
    def __init__(self, binary, verbose=False):
        self.binary = binary
        self.verbose = verbose
    
    def get_literal_value(self):
        num = ''
        for i in range(6, len(self.binary), 5):
            first = self.binary[i]
            num += self.binary[i+1:i+5]
            if first != '1':
                # this was the last number
                break
        return int(num,2), i+5


    def parse_until_len(self):
        start = 7 + 15
        stop = int(self.binary[7:start],2)
        if self.verbose: print('total len bits',stop)
        pnt = 0
        answers = []
        while pnt < stop:
            packet = Packet(self.binary[start + pnt : start + stop])
            res, l = packet.parse_bin()
            if self.verbose: print('res received', res, l)
            answers.append(res)
            pnt += l
        pnt = start + stop
        return answers, pnt


    def parse_until_npackets(self):
        npackets = int(self.binary[7:7+11],2)
        packets = 0
        pnt = 7+11
        answers = []
        while packets < npackets:
            packet = Packet(self.binary[pnt:])
            res, subpnt = packet.parse_bin()
            pnt += subpnt
            packets += 1
            answers.append(res)
        return answers, pnt


    def parse_bin(self):
        self.version = int(self.binary[:3], 2)
        self.typeid = int(self.binary[3:6], 2)
        
        if self.typeid == 4:
            res, pnt = self.get_literal_value()

        else:
            length_typeid = self.binary[6]
            if length_typeid == '0':
                if self.verbose: print('operator packet with length')
                answers, pnt = self.parse_until_len()
            elif length_typeid == '1':
                if self.verbose: print('operator packet with packets expected')
                answers, pnt = self.parse_until_npackets()
            else:
                if self.verbose: print('other length_typeid not implemented')
                sys.exit()
                    
            match self.typeid:
                case 0:
                    res = sum(answers)
                case 1:
                    res = math.prod(answers)
                case 2:
                    res = min(answers)
                case 3:
                    res = max(answers)
                case 5:
                    res = 1 if answers[0] > answers[1] else 0
                case 6:
                    res = 1 if answers[0] < answers[1] else 0
                case 7:
                    res = 1 if answers[0] == answers[1] else 0
            
        return res, pnt

In [8]:
assert parse_hex('C200B40A82') == 3
assert parse_hex('04005AC33890') == 54
assert parse_hex('880086C3E88112') == 7
assert parse_hex('CE00C43D881120') == 9
assert parse_hex('D8005AC2A8F0') == 1
assert parse_hex('F600BC2D8F') == 0
assert parse_hex('9C005AC2F8F0') == 0
assert parse_hex('9C0141080250320F1802104A08') == 1


In [9]:
# part 2 only, I removed the use of global ans for part 1
s = 'E058F79802FA00A4C1C496E5C738D860094BDF5F3ED004277DD87BB36C8EA800BDC3891D4AFA212012B64FE21801AB80021712E3CC771006A3E47B8811E4C01900043A1D41686E200DC4B8DB06C001098411C22B30085B2D6B743A6277CF719B28C9EA11AEABB6D200C9E6C6F801F493C7FE13278FFC26467C869BC802839E489C19934D935C984B88460085002F931F7D978740668A8C0139279C00D40401E8D1082318002111CE0F460500BE462F3350CD20AF339A7BB4599DA7B755B9E6B6007D25E87F3D2977543F00016A2DCB029009193D6842A754015CCAF652D6609D2F1EE27B28200C0A4B1DFCC9AC0109F82C4FC17880485E00D4C0010F8D110E118803F0DA1845A932B82E200D41E94AD7977699FED38C0169DD53B986BEE7E00A49A2CE554A73D5A6ED2F64B4804419508B00584019877142180803715224C613009E795E58FA45EA7C04C012D004E7E3FE64C27E3FE64C24FA5D331CFB024E0064DEEB49D0CC401A2004363AC6C8344008641B8351B08010882917E3D1801D2C7CA0124AE32DD3DDE86CF52BBFAAC2420099AC01496269FD65FA583A5A9ECD781A20094CE10A73F5F4EB450200D326D270021A9F8A349F7F897E85A4020CF802F238AEAA8D22D1397BF27A97FD220898600C4926CBAFCD1180087738FD353ECB7FDE94A6FBCAA0C3794875708032D8A1A0084AE378B994AE378B9A8007CD370A6F36C17C9BFCAEF18A73B2028C0A004CBC7D695773FAF1006E52539D2CFD800D24B577E1398C259802D3D23AB00540010A8611260D0002130D23645D3004A6791F22D802931FA4E46B31FA4E4686004A8014805AE0801AC050C38010600580109EC03CC200DD40031F100B166005200898A00690061860072801CE007B001573B5493004248EA553E462EC401A64EE2F6C7E23740094C952AFF031401A95A7192475CACF5E3F988E29627600E724DBA14CBE710C2C4E72302C91D12B0063F2BBFFC6A586A763B89C4DC9A0'
assert parse_hex(s) == 1510977819698